In [19]:
import os
import numpy as np
import argparse
from pathlib import Path
import open3d as o3d
from ppm import Ppm
from typing import Optional, Dict
import zarr
import dask
import dask.array as da
from tqdm import tqdm
from dask_image.ndmorph import binary_closing
from dask.diagnostics import ProgressBar
import fastmorph
import time
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

In [20]:
def nonzero_chunks_zarr(zarr_path, chunk_size=256):

    # zarr_array = zarr.open_array(zarr_path)# Load zarr as dask array
    # z_array = zarr.open(zarr_path, mode='r')
    chunks = (chunk_size,chunk_size,chunk_size)
    stime = time.time()
    data = da.from_zarr(zarr_path, chunks=chunks)
    print(f"Time to load zarr: {time.time() - stime:.2f}s")
    print(f"Data shape: {data.shape}")
    print(f"Data chunks: {data.chunks}")
    print(f"Data dtype: {data.dtype}")
    print(f"Number of chunks: {data.npartitions}")
    # print(f"Memory usage: {data.nbytes / 1e9:.2f} GB")    

    # Define function to sum each block
    def chunk_nonzero(block):
        # Return array with same number of dimensions but size 1 in each dimension
        return np.array([np.any(block)]).reshape((1,) * block.ndim)
    
    # Map the sum operation across all chunks and compute
    with ProgressBar():
        chunk_nonzero = data.map_blocks(chunk_nonzero, dtype=data.dtype).compute()

    return chunk_nonzero

In [21]:
zarr_path = '/Users/jamesdarby/Documents/VesuviusScroll/GP/labels_2D_to_3D/s1_791um_label.zarr'
chunk_size = 512
chunk_nonzero = nonzero_chunks_zarr(zarr_path, chunk_size)
#58s 1024 chunks nonzero
#50s 512 chunks nonzero
#42s 256 chunks nonzero???
#128 chunks takes forever


Time to load zarr: 0.00s
Data shape: (14376, 8096, 7888)
Data chunks: ((512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 40), (512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 416), (512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 208))
Data dtype: uint8
Number of chunks: 7424


In [22]:
print(chunk_nonzero.shape)
print(chunk_nonzero.sum())
print(chunk_nonzero.dtype)

(29, 16, 16)
88
bool


In [23]:
chunks = (chunk_size,chunk_size,chunk_size)
stime = time.time()
data = da.from_zarr(zarr_path, chunks=chunks)
z,y,x = np.where(chunk_nonzero)[0:3]
z,y,x = z[10], y[10], x[10]
chunk = data.blocks[z,y,x]
chunk.compute()

print(chunk.shape, chunk)
print(f"Time to load zarr: {time.time() - stime:.2f}s")

(512, 512, 512) dask.array<blocks, shape=(512, 512, 512), dtype=uint8, chunksize=(512, 512, 512), chunktype=numpy.ndarray>
Time to load zarr: 0.17s


/Users/jamesdarby/anaconda3/envs/2d3d/lib/python3.12/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [24]:
# Set middle planes to 128
z_mid = chunk.shape[0] // 2
y_mid = chunk.shape[1] // 2
x_mid = chunk.shape[2] // 2

chunk[z_mid,:,:] = 128  # z middle plane
chunk[:,y_mid,:] = 128  # y middle plane 
chunk[:,:,x_mid] = 128  # x middle plane


In [25]:
plot_data = chunk
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(plot_data[:,slice_index,:], cmap='gray')
    elif axis == 2:
        plt.imshow(plot_data[:,:,slice_index], cmap='gray')
    else:
        plt.imshow(plot_data[slice_index,:,:], cmap='gray')
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=plot_data.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=1))

interactive(children=(IntSlider(value=0, description='slice_index', max=511), IntSlider(value=1, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [26]:
radius = 16
arr = chunk
arr = np.pad(arr, pad_width=radius, mode='constant', constant_values=0)
stime = time.time()
arr = fastmorph.dilate(arr, parallel=8)
arr = fastmorph.erode(arr, parallel=8)
closed_chunk = arr[radius:-radius,radius:-radius,radius:-radius]
print(f"Time to close: {time.time() - stime:.4f}s")

plot_data = closed_chunk
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(plot_data[:,slice_index,:], cmap='gray')
    elif axis == 2:
        plt.imshow(plot_data[:,:,slice_index], cmap='gray')
    else:
        plt.imshow(plot_data[slice_index,:,:], cmap='gray')
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()
interact(plot_slice, slice_index=IntSlider(min=0, max=plot_data.shape[0]-1, step=1, value=radius+1), axis=IntSlider(min=0, max=2, step=1, value=1))

Time to close: 0.8644s


interactive(children=(IntSlider(value=17, description='slice_index', max=511), IntSlider(value=1, description=…

<function __main__.plot_slice(slice_index, axis=0)>

In [27]:
radius = 25

arr = np.array((chunk > 0))

arr = np.pad(arr, pad_width=radius, mode='constant', constant_values=1)
print(arr.shape)

arr = fastmorph.dilate(arr, mode=fastmorph.Mode.grey)
plot_data = arr
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(plot_data[:,slice_index,:], cmap='gray')
    elif axis == 2:
        plt.imshow(plot_data[:,:,slice_index], cmap='gray')
    else:
        plt.imshow(plot_data[slice_index,:,:], cmap='gray')
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()
interact(plot_slice, slice_index=IntSlider(min=0, max=plot_data.shape[0]-1, step=1, value=radius+1), axis=IntSlider(min=0, max=2, step=1, value=1))

(562, 562, 562)


interactive(children=(IntSlider(value=26, description='slice_index', max=561), IntSlider(value=1, description=…

<function __main__.plot_slice(slice_index, axis=0)>

In [28]:
# Pad chunk on all sides
stime = time.time()
radius = 16 #any radius lower than 16 seems to clip the borders into the data???
arr = np.array((chunk > 0))
# arr = arr[0:256,0:256,0:256]

arr = np.pad(arr, pad_width=radius, mode='constant', constant_values=0)
print(arr.dtype)
print(arr.shape)
arr = fastmorph.dilate(arr, parallel=8)
arr = fastmorph.erode(arr, parallel=8)
#1.8s 512 para 1

# arr = fastmorph.closing(arr, parallel=2) 
#.28s 256 para 1
#.24s 256 para 2
#1.8s 512 para 1
#1.2s 512 para 2
#.88s 512 para 4

closed_chunk = arr[radius:-radius,radius:-radius,radius:-radius]
print(closed_chunk.shape, closed_chunk.dtype)
print(f"Time to close: {time.time() - stime:.4f}s")

bool
(544, 544, 544)
(512, 512, 512) bool
Time to close: 1.0333s


In [29]:
plot_data = closed_chunk
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(plot_data[:,slice_index,:], cmap='gray')
    elif axis == 2:
        plt.imshow(plot_data[:,:,slice_index], cmap='gray')
    else:
        plt.imshow(plot_data[slice_index,:,:], cmap='gray')
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

# Create a slider to browse through slices
interact(plot_slice, slice_index=IntSlider(min=0, max=plot_data.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=1))

interactive(children=(IntSlider(value=0, description='slice_index', max=511), IntSlider(value=1, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [30]:
# Get zyx block coordinates of nonzero blocks from the 3d mask array
nonzero_coords = list(zip(*np.where(chunk_nonzero)))
print(f"Found {len(nonzero_coords)} nonzero blocks")
print("First 5 block coordinates (z,y,x):", nonzero_coords[:5])


Found 88 nonzero blocks
First 5 block coordinates (z,y,x): [(19, 4, 3), (19, 4, 4), (19, 4, 5), (19, 4, 6), (19, 4, 7)]


In [42]:
def morph_close_chunks_zarr(zarr_path, chunk_size=512, radius=16, parallel=1, zarr_name="closed_label.zarr", morph_labels=[]):
    chunks = (chunk_size,chunk_size,chunk_size)
    data = da.from_zarr(zarr_path, chunks=chunks)

    def morph_close_chunk(block):
        # if morph labels is None or empty, process blocks with any nonzero values
        if morph_labels is None or not morph_labels:
            if not np.any(block):
                return block
        elif len(morph_labels) == 1:
            if not (block == morph_labels[0]).any():
                return block
        else: 
            if not np.isin(block, morph_labels).any():
                return block
            
        arr = np.array(block)  
        arr = np.pad(arr, pad_width=radius, mode='constant', constant_values=0)
        arr = fastmorph.dilate(arr, parallel=parallel)
        arr = fastmorph.erode(arr, parallel=parallel)
        return arr[radius:-radius,radius:-radius,radius:-radius]

    
    closed_data = data.map_blocks(morph_close_chunk, dtype=data.dtype, meta=True)#.compute(scheduler='processes')

    with ProgressBar():
        closed_data.to_zarr(zarr_name, compute=False, overwrite=True).compute(scheduler='processes')

zarr_name = "closed_s1_791um_label.zarr"
morph_labels = [] #pass empty list to morph all >0 values, specify list of labels to morph only those labels
morph_close_chunks_zarr(zarr_path, chunk_size, radius, parallel=1, zarr_name=zarr_name, morph_labels=morph_labels)


#90s to morph with no morph_labels (morph all >0 values); skip with .any()
#2m18s to check single label with arr==val, .any()
#3m6s to check multiple labels with np.isin

[########################################] | 100% Completed | 98.40 s
